In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DMT_PRD")
dbutils.widgets.text("target_table", "a_daily_scct_store_sku_drdio")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_DB_INVENTORY.D_MDQ_STORE_DC_SKU_DAILY,VCM_DB_INVENTORY.D_PROBLEM_ARTICLE,VCM_DWH_PRD.D_ASSORTMENT_ACTIVE, VCM_DWH_PRD.D_CORE_RANGE_ASSORTMENT_WMP,VCM_DWH_PRD.D_MDQ_AMBIENT_BIWEEKLY,VCM_DWH_PRD.D_MDQ_AMBIENT_DAILY,VCM_DWH_PRD.D_MDQ_POG_STORE_SKU,VCM_DWH_PRD.D_MIN_STOCK,VCM_DWH_PRD.D_PRODUCT,VCM_DWH_PRD.D_SALE_PRICE_BASE_UNIT_DAILY,VCM_DWH_PRD.D_STORE,VCM_DWH_PRD.D_UNIT_CONV,VCM_DWH_PRD.F_COGS_90D"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dmt.a_daily_scct_store_sku_drdio (
  calday DATE,
  isoweek BIGINT,
  type STRING,
  store_id STRING,
  store_status_desc STRING,
  product_id STRING,
  mdq DOUBLE,
  internal_report_mdq DOUBLE,
  mdq_type STRING,
  rr DOUBLE,
  min_stock DOUBLE,
  replenishment_mode STRING,
  dc_id STRING,
  mdq_age BIGINT,
  mch2_id STRING,
  core_range STRING,
  order_method STRING,
  is_problem_article STRING,
  is_block STRING,
  target_dio DOUBLE,
  rev_amt DOUBLE,
  cogs DOUBLE,
  closing_stock_qty DOUBLE,
  closing_stock_value DOUBLE,
  cogs_30d DOUBLE,
  sale_qty_30d DOUBLE,
  rev_amt_30d DOUBLE,
  opening_stock_qty DOUBLE,
  po_pending_qty DOUBLE,
  po_pending_val DOUBLE,
  dr_count BIGINT,
  dr_sum BIGINT,
  adj_dr BIGINT,
  stock_excess DOUBLE,
  stock_shortage DOUBLE,
  stock_excess_val DOUBLE,
  stock_shortage_val DOUBLE,
  lost_sale DOUBLE,
  lost_sale_value DOUBLE,
  winare_project STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
  spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_a_daily_scct_store_sku_drdio
    AS 
    WITH asm AS(
      SELECT a.* EXCEPT(priority)
      FROM (
        SELECT
          a.calday,
          a.store_id,
          a.product_id,
          mch2_id,
          'ACTIVE ASM' type,
          '1' priority
        FROM
          {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his a
          INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store USING (store_id)
          INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product USING (product_id)
        WHERE
          a.CALDAY IN ({CALDAY_IN_STR})
          AND business_unit IN ('1500','2000')
          AND mch2_id IN ('101','102','201','202','203')
        UNION ALL
        SELECT DISTINCT
          calday,
          store_id,
          product_id,
          mch2_id,
          'HAVING ACTUAL DATA' type,
          '2' priority
        FROM
          {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
          INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store USING (store_id)
          INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product USING (product_id)
        WHERE
          CALDAY IN ({CALDAY_IN_STR})
          AND business_unit IN ('1500','2000')
          AND mch2_id IN ('101','102','201','202','203')
          AND (revenue.base_sale_qty <> 0 OR revenue.rev_amt <> 0 OR revenue.cogs <> 0
          OR stock.closing_stock_value <> 0 OR stock.closing_stock_quantity <> 0)
        UNION ALL
        SELECT DISTINCT
          calday,
          store_id,
          product_id,
          mch2_id,
          'HAVING COGS30D' type,
          '3' priority
        FROM
          {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs_90d
          INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store USING (store_id)
          INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product USING (product_id)
        WHERE
          CALDAY IN ({CALDAY_IN_STR})
          AND business_unit IN ('1500','2000')
          AND mch2_id IN ('101','102','201','202','203')
          AND cogs_30d <> 0
      ) a
      QUALIFY ROW_NUMBER() OVER (PARTITION BY calday, store_id, product_id ORDER BY priority) = 1
    ),
    store_check AS (
      SELECT
        store_id,
        store_name,
        store_status_desc
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
      WHERE (
        UPPER(store_name) LIKE '%DO%NOT%USE%'
        OR UPPER(store_name) LIKE '[BLOCK]%'
        OR UPPER(store_name) LIKE '%B2B%'
        OR UPPER(store_name) LIKE '%OUTLET%'
        OR UPPER(store_name) LIKE '%ONLIN%'
        OR UPPER(store_name) LIKE '%HUB%'
        OR UPPER(store_id) LIKE 'GH%'
      ) AND business_unit IN ('1500', '2000')
    ),
    dynamic_assortment AS (
      SELECT DISTINCT
        calday,
        store_id,
        product_id
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_dynamic_assorment_his
      WHERE is_keeping = 0 
      AND calday between '2025-04-14' and DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
    ),
    winare_dsd_store_sku AS (
      SELECT DISTINCT
        product_id,
        store_id,
        calday
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_dsd_golive_his
      WHERE UPPER(golive) = 'YES'
      QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id, product_id, calday ORDER BY calday) = 1
    ),
    winare_fresh_store_sku AS (
      SELECT DISTINCT
        a.calday store_list_cutoff_day,
        c.calday,
        store_id
      FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group_his a
      INNER JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_time WHERE dayofweek = 5) b ON a.calday = b.calday
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time c ON DATE_ADD(TRUNC(a.calday, 'IW'), 7) = c.calmonday
    ),
    winare_mml_store_sku AS (
      SELECT DISTINCT
        store_id,
        product_id,
        a.current_date calday
      FROM {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli_his a
    ),
    --DC_MODE_INFO--MAPPING INFORMATION OF DC REPLENISHMENT MODE (MDQ,MDQ_AGE)
    all_store_sku AS (
      SELECT
        a.*,
        st.business_unit,
        CASE
          WHEN st.business_unit = '1500' THEN 'Supermarket'
          WHEN IFNULL(st.concept,'') IN ('Rural','WiN') THEN st.concept
          ELSE 'Urban'
        END concept,
        IFNULL(b.internal_report_mdq,0) internal_report_mdq,
        IFNULL(b.mdq,0) mdq,
        CASE
          WHEN d.dc_id IS NOT NULL THEN 'DC'
          WHEN a.type <> 'ACTIVE ASM' THEN 'NA'
          ELSE 'DSD'
        END replenishment_mode,
        IFNULL(
          IFNULL(
            b.assortment_type,
            CASE
              WHEN st.business_unit = '1500' THEN asm_3.core_range
              WHEN st.business_unit = '2000' THEN asm_2.core_range
              ELSE 'NA'
            END
          ), 'NA') core_range,
        b.mdq_age,
        d.dc_id,
        d.is_problem_article,
        st_his.store_status_desc,
        CASE
          WHEN d.dc_id IS NOT NULL THEN 'DC'
          WHEN wd.store_id IS NOT NULL THEN 'DSD'
          WHEN wf.store_id IS NOT NULL THEN 'WINFRESH'
          WHEN st.business_unit = '2000' AND pr.mch5_id = '102050404' THEN 'TNXX'
          WHEN wm.store_id IS NOT NULL THEN 'MML'
          ELSE 'OTHER'
        END winare_project
      FROM
        asm a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st ON a.store_id = st.store_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product pr ON a.product_id = pr.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_hist st_his ON a.store_id = st_his.store_id and a.calday = st_his.calday
        LEFT JOIN (
          SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly_hist WHERE calday >= DATE({START_DATE})  AND calday < '2024-08-08'
          UNION ALL
          SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist WHERE calday >= DATE({START_DATE}) AND calday >='2024-08-08'
        ) b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id
        LEFT JOIN (
          SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his
          WHERE calday in ({CALDAY_IN_STR})
          QUALIFY ROW_NUMBER() OVER(PARTITION BY store_id, product_id, calday ORDER BY dc_id) = 1
        ) d ON a.store_id = d.store_id AND a.product_id = d.product_id  AND a.calday = d.calday
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_wmp asm_2 ON a.store_id = asm_2.store_id AND a.product_id = asm_2.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_wmt asm_3 ON a.store_id = asm_3.store_id AND a.product_id = asm_3.product_id
        --MAPPING LUỒNG WINARE
        LEFT JOIN winare_dsd_store_sku wd ON a.calday = wd.calday AND a.store_id = wd.store_id AND a.product_id = wd.product_id
        LEFT JOIN winare_fresh_store_sku wf ON a.calday = wf.calday AND a.store_id = wf.store_id AND pr.mch3_id = '10103'
        LEFT JOIN winare_mml_store_sku wm ON a.calday = wm.calday AND a.store_id = wm.store_id AND a.product_id = wm.product_id
    ),
    min_stock_tmp AS (
      SELECT
        a.* ,
        CASE
          WHEN c.approval_status = 'YES' THEN 'APPROVED MDQ_POG'
          WHEN c.approval_status = 'IN REVIEW' THEN 'IN REVIEW MDQ_POG'
          ELSE 'OLD_MDQ'
        END mdq_type,
        IFNULL(b.rr,0) rr
      FROM
        all_store_sku a
        LEFT JOIN (
          SELECT
            calday,
            store_id,
            product_id,
            SUM(sell_rate) rr
          FROM {catalog_name}.udp_wcm_gold_vcm_dmt.f_corrected_sellrate -- Error of DWH 
          WHERE calday IN ({CALDAY_IN_STR})
          GROUP BY 1,2,3
        ) b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku c
        ON a.product_id = c.product_id AND a.store_id = c.store_id
    ),
    min_stock AS (
      SELECT
        a.*,
        GREATEST(internal_report_mdq,rr) min_stock 
      FROM min_stock_tmp a
    ),
    sale_stock_daily AS (
      SELECT
        a.calday,
        a.store_id,
        a.product_id,
        IFNULL(IF(business_unit = '1500',b.revenue.rev_amt,b.revenue.rev_amt_pos), 0) rev_amt,
        IFNULL(b.revenue.base_sale_qty, 0) base_sale_qty,
        IFNULL(b.revenue.cogs_pos, 0) cogs,
        IFNULL(b.stock.closing_stock_quantity_sloc_1000, 0) closing_stock_qty,
        IFNULL(b.stock.closing_stock_value, 0) closing_stock_value
      FROM
        min_stock a
        LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily  WHERE calday >= DATE_SUB({START_DATE}, 30)) b
        ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id
      WHERE a.calday <= date_sub(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)
    ),
    sum_30d AS (
      SELECT
        a.calday,
        a1.store_id,
        a1.product_id,
        IFNULL(stock.closing_stock_quantity_sloc_1000, 0) closing_stock_qty,
        SUM(IFNULL(revenue.cogs_pos, 0)) OVER (PARTITION BY a1.store_id, a1.product_id ORDER BY a.calday ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) cogs_30d,
        SUM(IFNULL(revenue.base_sale_qty, 0)) OVER (PARTITION BY a1.store_id,a1.product_id ORDER BY a.calday ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) sale_qty_30d,
        SUM(IFNULL(IF(c.business_unit = '1500',b.revenue.rev_amt, b.revenue.rev_amt_pos), 0)) OVER (PARTITION BY a1.store_id,a1.product_id ORDER BY a.calday ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) rev_amt_30d
      FROM
        (SELECT EXPLODE(SEQUENCE(DATE_SUB(DATE{START_DATE}, 30), DATE{END_DATE}, INTERVAL 1 DAY)) AS CALDAY) A,
        (SELECT DISTINCT store_id, product_id FROM sale_stock_daily) a1
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily  b ON a.calday = b.calday AND a1.store_id = b.store_id AND a1.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a1.store_id = c.store_id
      WHERE a.calday >= DATE_SUB({START_DATE}, 30)
    ),
    sale_30d AS (
      SELECT
        a.*,
        b.cogs_30d,
        b.sale_qty_30d,
        b.rev_amt_30d
      FROM
        sale_stock_daily a
        LEFT JOIN sum_30d b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id
    ),
    cogs_val_30d AS (
      SELECT
        calday,
        store_id,
        product_id,
        SUM(cogs_30d) cogs_30d
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs_90d
      WHERE calday IN ({CALDAY_IN_STR})
      GROUP BY 1,2,3
    ),
    po_pending_qty AS (
      SELECT 
        calday,
        store_id,
        product_id,
        SUM(a.open_order) po_pending_qty
      FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_winare_dc_open_order_store_his a
      WHERE calday IN ({CALDAY_IN_STR})
      GROUP BY 1,2,3
    ),
    po_pending_val AS (
      SELECT
        DATE_ADD(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), -1) calday,
        a.*
      FROM (
        SELECT
          a.store_id,
          a.product_id,
          IFNULL(SUM(a.po_value), 0) po_pending_val
        FROM
          {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
          INNER JOIN (SELECT DISTINCT store_id,product_id FROM all_store_sku) b ON a.store_id = b.store_id AND a.product_id = b.product_id
          LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
        WHERE
          a.creation_date >= DATE(date_sub(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 21)) AND a.creation_date < DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
          AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
          AND (a.po_gr_date IS NULL)
          AND (a.gr_date IS NULL)
          AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng', 'Quá hạn') OR a.webportal_status IS NULL)
        GROUP BY 1,2
      ) a
    ),
    summary AS (
      SELECT
        a.calday,
        c.isoweek,
        a.type,
        a.store_id,
        a.store_status_desc,
        a.product_id,
        a.mdq,
        a.internal_report_mdq,
        a.mdq_type,
        IFNULL(a.rr,0) rr,
        a.min_stock,
        a.replenishment_mode,
        a.dc_id,
        a.mdq_age,
        a.mch2_id,
        a.core_range,
        i.order_method,
        CASE
          WHEN is_problem_article IS NOT NULL THEN is_problem_article
          ELSE IF(g.sku IS NULL,'NO','YES')
        END is_problem_article,
        IF(h.store_id IS NULL,'NO','YES') is_block,
        CASE
          WHEN a.mch2_id = '101' THEN 13.2
          WHEN a.mch2_id = '102' THEN 49.6
          WHEN a.mch2_id IN ('201','202','203') THEN 96.3
        END target_dio,
        IFNULL(d.rev_amt, 0) rev_amt,
        IFNULL(d.cogs, 0) cogs,
        IFNULL(d1.closing_stock_qty, 0) closing_stock_qty,
        IFNULL(d1.closing_stock_value, 0) closing_stock_value,
        IFNULL(e1.cogs_30d, 0) cogs_30d,
        IFNULL(e.sale_qty_30d, 0) sale_qty_30d,
        IFNULL(e.rev_amt_30d, 0) rev_amt_30d,
        IFNULL(d2.closing_stock_qty,0) opening_stock_qty,
        IFNULL(b2.po_pending_qty, 0) po_pending_qty,
        IFNULL(b1.po_pending_val, 0) po_pending_val,
        IF((k1.store_id IS NOT NULL) OR (k2.store_id IS NOT NULL) OR (k3.store_id IS NOT NULL),
            0,
            IF(a.min_stock>0 AND type = 'ACTIVE ASM' AND mch2_id <> '101' and a.store_status_desc = 'Đang hoạt động',1,0) 
          ) dr_count,
        IF((k1.store_id IS NOT NULL) OR (k2.store_id IS NOT NULL) OR (k3.store_id IS NOT NULL),
            0,
            IF(a.min_stock>0 AND type = 'ACTIVE ASM' AND mch2_id <> '101' and a.store_status_desc = 'Đang hoạt động' AND IFNULL(d1.closing_stock_qty,0)>=min_stock,1,0) 
          ) dr_sum,
        IF((k1.store_id IS NOT NULL) OR (k2.store_id IS NOT NULL) OR (k3.store_id IS NOT NULL),
            0,
            IF(a.min_stock>0 AND type = 'ACTIVE ASM' AND mch2_id <> '101' and a.store_status_desc = 'Đang hoạt động' AND (IFNULL(d1.closing_stock_qty,0) + IFNULL(b2.po_pending_qty, 0))>=a.min_stock,1,0)
          ) adj_dr,
          winare_project
      FROM
        min_stock a
        LEFT JOIN po_pending_val b1 on a.calday = b1.calday AND a.product_id = b1.product_id AND a.store_id = b1.store_id
        LEFT JOIN po_pending_qty b2 on a.calday = b2.calday AND a.product_id = b2.product_id AND a.store_id = b2.store_id
        INNER JOIN {catalog_name}.udp_wcm_silver_dim.d_time c ON a.calday = c.calday
        LEFT JOIN sale_stock_daily d ON a.product_id = d.product_id AND a.store_id = d.store_id AND a.calday = d.calday
        LEFT JOIN sale_stock_daily d1 ON a.product_id = d1.product_id AND a.store_id = d1.store_id AND a.calday = d1.calday
        LEFT JOIN sum_30d d2 ON a.product_id = d2.product_id AND a.store_id = d2.store_id AND a.calday = d2.calday+1
        LEFT JOIN sale_30d e ON a.product_id = e.product_id AND a.store_id = e.store_id AND a.calday = e.calday
        LEFT JOIN cogs_val_30d e1 ON a.product_id = e1.product_id AND a.store_id = e1.store_id AND a.calday = e1.calday
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st ON a.store_id = st.store_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_problem_article_his g
          ON st.business_unit = g.business_unit AND st.region_vn = g.area AND a.calday = g.calday AND a.product_id = g.sku
        LEFT JOIN (
          SELECT * FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku_current_his a
          WHERE a.calday IN ({CALDAY_IN_STR}) and reason like "High stock%"
        ) h
          ON a.calday = h.calday + 1 AND a.product_id = h.product_id AND a.store_id = h.store_id
        LEFT JOIN (
          SELECT
            store_id, article,
            order_method,
            start_date,
            end_date
          FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_min_stock
        ) i ON
          a.store_id = i.store_id
          AND a.product_id = i.article
          AND i.start_date <= a.calday
          AND i.end_date >= a.calday
        LEFT JOIN (SELECT * FROM dynamic_assortment WHERE calday = '2025-04-14') k1 ON a.store_id = k1.store_id and a.product_id = k1.product_id and (a.calday <= '2025-04-14')
        LEFT JOIN (SELECT * FROM dynamic_assortment WHERE calday = '2025-04-14') k2 ON a.store_id = k2.store_id and a.product_id = k2.product_id and a.calday = k2.calday
        LEFT JOIN store_check k3 ON a.store_id = k3.store_id
    ),
    main AS (
      SELECT
        a.*,
        CASE
          WHEN mch2_id = '101' THEN 0
          ELSE IF((opening_stock_qty - rr * target_dio - mdq) < 0 ,0 ,(opening_stock_qty - rr * target_dio - mdq))
        END stock_excess,
        CASE
          WHEN mch2_id = '101' THEN 0
          ELSE IF((opening_stock_qty - rr - mdq) <=0 AND dr_count = 1,(opening_stock_qty - rr - mdq),0)
        END stock_shortage
      FROM summary a
    )
    SELECT
      a.* EXCEPT(winare_project), 
      CASE 
        WHEN closing_stock_qty != 0 THEN stock_excess * (closing_stock_value / closing_stock_qty)
        ELSE 0 
      END AS stock_excess_val,
      CASE 
        WHEN closing_stock_qty != 0 THEN stock_shortage * (closing_stock_value / closing_stock_qty)
        ELSE 0 
      END AS stock_shortage_val,
      IF(mch2_id ='101',0,IF(rr-greatest(0,opening_stock_qty)<0,0,rr-greatest(0,opening_stock_qty))) lost_sale,
      IF(mch2_id = '101',0,IFNULL(d.sale_price_exc_vat,0) * IF(rr-greatest(0,opening_stock_qty)<0,0,rr-greatest(0,opening_stock_qty))) lost_sale_value,
      winare_project
    FROM
      main a
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_base_unit_daily d ON a.calday = d.calday AND a.product_id = d.product_id AND a.store_id = d.store_id
    WHERE a.calday <= date_sub(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)
    """)
else:
  print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
# Get the calday list string from temp table
calday_query = """
SELECT
    CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
FROM temp_a_daily_scct_store_sku_drdio
"""

calday_df = spark.sql(calday_query)
calday_in_str = calday_df.collect()[0][0]

# Only run DELETE if the string is non-empty
if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
    spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_daily_scct_store_sku_drdio
    WHERE calday IN ({calday_in_str})
    """)
else:
    print("No valid CALDAYs found to delete.")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dmt.a_daily_scct_store_sku_drdio").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dmt.a_daily_scct_store_sku_drdio ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dmt.a_daily_scct_store_sku_drdio can't be inserted using calday's from temp_{target_table}"
    )